In [1]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html -q

In [3]:
import jax
print('JAX is running on', jax.lib.xla_bridge.get_backend().platform)
print(jax.devices())

JAX is running on tpu
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


In [4]:
%%capture
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git

!pip install flax
!pip install git+https://github.com/deepmind/optax.git

In [5]:
!pip install --upgrade jax jaxlib optax jax-dataloader -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 43.3 MB/s eta 0:00:00


In [6]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import flax
from jax import jit, random
import optax
from transformers import MarianTokenizer, FlaxMarianMTModel, AutoConfig
from datasets import load_dataset, DatasetDict
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from functools import partial
from tqdm import tqdm
import numpy as np
from time import time
from typing import Callable
import jax_dataloader as jdl

In [7]:
def get_config():
    config = {
        'model_name': 'Helsinki-NLP/opus-mt-en-hu', # 't5-small',
        'max_length': 64,
        'batch_size': 64,
        'lr': 10 ** -5,
        'epochs': 10,
        'seed': 42,
        'metric_name': 'sacrebleu',
        'per_device_batch_size': 64

    }
    return config

In [8]:
config = get_config()

In [9]:
# Load dataset and metric
dataset = load_dataset('Helsinki-NLP/opus_books', 'en-hu')
val_test_set = dataset['train'].train_test_split(test_size=0.2, seed=42)
test_set = val_test_set['test'].train_test_split(test_size=0.5, seed=42)

dataset = DatasetDict({
    'train': val_test_set['train'],
    'val': test_set['test'],
    'test': test_set['train']
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/137151 [00:00<?, ? examples/s]

In [10]:
tokenizer = MarianTokenizer.from_pretrained(config['model_name'])
# Load the model
model_config = AutoConfig.from_pretrained(config['model_name'])
model = FlaxMarianMTModel.from_pretrained(config['model_name'], config=model_config, from_pt=True)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/792k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/850k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-hu were not used when initializing FlaxMarianMTModel: {('model', 'decoder', 'embed_tokens', 'kernel'), ('model', 'encoder', 'embed_positions', 'kernel'), ('model', 'decoder', 'embed_positions', 'kernel'), ('model', 'encoder', 'embed_tokens', 'kernel')}
- This IS expected if you are initializing FlaxMarianMTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxMarianMTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:
def preprocess_fn(dataset: DatasetDict):
    inputs = [ex['en'] for ex in dataset['translation']]
    targets = [ex['hu'] for ex in dataset['translation']]
    model_inputs = tokenizer(inputs, max_length=config['max_length'], truncation=True, padding='longest')

    # tokenizer targets
    #with tokenizer.as_target_tokenizer():
    labels =  tokenizer(targets, max_length=config['max_length'], truncation=True, padding='longest')

    model_inputs['labels'] = labels.input_ids

    return model_inputs

In [12]:
tokenized_datasets = dataset.map(preprocess_fn, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/109720 [00:00<?, ? examples/s]

Map:   0%|          | 0/13716 [00:00<?, ? examples/s]

Map:   0%|          | 0/13715 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 109720
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13716
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13715
    })
})

### Dataloader

In [14]:
total_batch_size = config['per_device_batch_size'] * 1
print("The overall batch size (both for training and eval) is", total_batch_size)

The overall batch size (both for training and eval) is 64


In [15]:
train_loader = jdl.DataLoader(tokenized_datasets['train'], 'jax', batch_size=total_batch_size, shuffle=True)
val_loader = jdl.DataLoader(tokenized_datasets['val'], 'jax', batch_size=total_batch_size, shuffle=False, drop_last=True)
test_loader = jdl.DataLoader(tokenized_datasets['test'], 'jax', batch_size=total_batch_size, shuffle=False)

In [16]:
len_train_loader = len(train_loader)
len_val_loader = len(val_loader)

### Tokenizer and model

In [17]:
num_train_steps = len(dataset['train']) // total_batch_size * config['epochs']

#learning_rate_function = optax.linear_schedule(init_value=config['lr'], end_value=0, transition_steps=num_train_steps)
learning_rate_function = optax.cosine_onecycle_schedule(transition_steps=num_train_steps, peak_value=config['lr'], pct_start=0.1, )

In [18]:
class TrainState(train_state.TrainState):
    loss_function: Callable = flax.struct.field(pytree_node=False)

In [19]:
@jit
def loss_function(logits, labels):
    # Compute the cross-entropy loss
    loss = optax.softmax_cross_entropy(logits, onehot(labels, num_classes=logits.shape[-1]))
    return jnp.mean(loss)

In [20]:
state = TrainState.create(
    apply_fn=model.__call__,
    params=model.params,
    tx=optax.adamw(learning_rate=learning_rate_function),
    loss_function=loss_function,
)

In [22]:
@jit
def train_step(state, batch, dropout_rng):
    dropout_rng, new_dropout_rng = random.split(dropout_rng)
    def compute_loss(params):
        labels = batch.pop("labels")
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = state.loss_function(logits, labels)
        return loss

    loss, grads = jax.value_and_grad(compute_loss)(state.params)
    new_state = state.apply_gradients(grads=grads)
    return new_state, loss, new_dropout_rng

In [23]:
@jit
def eval_step(state, batch):
    labels = batch.pop("labels")
    logits = state.apply_fn(**batch, params=state.params, train=False)[0]
    loss = state.loss_function(logits, labels)
    return loss

# Training

In [24]:
rng = jax.random.PRNGKey(0)
dropout_rngs = rng

In [25]:
total_batch_size = config['per_device_batch_size'] * 1 # Training in one machine tpu
print("The overall batch size (both for training and eval) is", total_batch_size)

start_time = time()
print('======== Start training ======== ')
for epoch in range(config['epochs']):

    training_loss = 0
    with tqdm(total=len_train_loader, desc="Training...", leave=False) as progress_bar_train:
        for batch in train_loader:
            state, loss, dropout_rngs = train_step(state, batch, dropout_rngs)
            training_loss += jax.device_get(loss)
            progress_bar_train.update(1)
        training_loss = training_loss / len_train_loader
        progress_bar_train.write(
              f"Train... ({epoch}/{config['epochs']} | Loss: {round(training_loss, 3)})"
        )

    eval_loss = 0
    with tqdm(total=len_val_loader, desc="Evaluating...", leave=False) as progress_bar_eval:
        for batch in val_loader:
            loss = eval_step(state, batch)
            eval_loss += jax.device_get(loss)
            progress_bar_eval.update(1)
        eval_loss = eval_loss / len_val_loader
        progress_bar_eval.write(
            f"Eval... ({epoch}/{config['epochs']} | Loss: {eval_loss})"
        )


print('======== End training ========')
total_training_time = time() - start_time
print(f"Total training time: {total_training_time}")

The overall batch size (both for training and eval) is 64
======== Start training ======== 


Train... (0/10 | Loss: 4.798)


Eval... (0/10 | Loss: 3.8515882302667492)


Train... (1/10 | Loss: 3.813)


Eval... (1/10 | Loss: 3.735729615264964)


...

Eval... (7/10 | Loss: 3.5760628408360704)


Train... (8/10 | Loss: 3.546)


Eval... (8/10 | Loss: 3.576825897270274)


Train... (9/10 | Loss: 3.544)


Eval... (9/10 | Loss: 3.5760812102077164)
======== End training ========
Total training time: 2990.2118105888367
